In [ ]:
from ultralytics import YOLO
import os
import torch
import shutil
import glob

# --- 1. 환경 확인 및 설정 ---
print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("현재 사용 중인 장치: CPU")


# --- 2. 모델 학습 ---
def train_yolo_on_sagemaker():
    # 2-1. 모델 로드
    # yolov8n.pt는 가장 작고 빠른 모델입니다.
    model = YOLO('yolov8n.pt')
    # 2-3. 모델 학습 실행
    results = model.train(
        data='./data.yaml',
        epochs=50,
        imgsz=640,
        project='result/detect',  # 결과는 노트북 인스턴스 내에 저장됩니다.
        name='skin_disease_sagemaker_run'
    )
    
    print("\n모델 학습이 완료되었습니다!")
    print(f"학습 결과는 노트북 인스턴스의 '{results.save_dir}' 폴더에 저장되었습니다.")
    
    # --- 2-5. 결과물을 S3에 업로드 ---
    result_folder_name = os.path.basename(results.save_dir)
    s3_output_path = f"s3://say1-5team-bucket/yolo-results/{result_folder_name}"
    
    print(f"학습 결과물을 S3에 백업합니다: {s3_output_path}")
    
    os.system(f"aws s3 cp --recursive {results.save_dir} {s3_output_path}")
    print("S3 백업 완료!")


# --- 3. 스크립트 실행 ---
if __name__ == '__main__':
    train_yolo_on_sagemaker()